In [1]:
import json
from matplotlib import pyplot as plt
from pprint import pprint
import numpy as np
import pandas as pd
import os
import math
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Preparing the data
===================

In [2]:

def getPVParams(track, mcps, mcvs):
    """ Given a track dict, the list of MCParticles and MCVertices 
    return the track PV position and ID (or None if not found or ambiguous)"""
    track_mcps = track['MCPs']
    # We ignore the tracks with no association to MCP or more than one
    if len(track_mcps) != 1:
        return None
    track_mcp = mcps[track_mcps[0]]
    track_pv_id = track_mcp['PV']
    track_pv = mcvs[track_pv_id]
    pos = track_pv['Pos']
    return pos[0], pos[1], pos[2], track_pv['key']



In [3]:

def getTrackFirstHitZ(track, hits):
    """ Returns the z of the first hit for the given track """
    hzs = [ hits[h]['z'] for h in track['LHCbIDs'] ]
    return min(hzs), max(hzs) 

 

In [4]:


def readData(filename, mycontainer):
    with open(filename) as mf:
        jdata = json.load(mf)
    #for key in jdata:
    #    print(key)
    VeloTracks  = jdata['VeloTracks']
    VeloHits  = jdata['VPClusters']
    mcps = jdata['MCParticles']
    mcvs = jdata['MCVertices']
    for  l in VeloTracks.values():
        if l['isBackwards']:
            continue
        pvps = getPVParams(l, mcps, mcvs)
        if pvps != None:
            mycontainer.append(l['ClosestToBeam'] + list(pvps) + [len(l['LHCbIDs'])] 
                             + [ *getTrackFirstHitZ(l, VeloHits)]  + [ jdata['EventNumber']])


def load_data(filelist, prefix="data/train"):

    container = []
    for fn in filelist:
        readData(os.path.join(prefix, fn), container)

    df = pd.DataFrame.from_records(container, columns=['x', 'y', 'z', 'tx', 'ty', 'qp', 'pvx', 'pvy', 'pvz',
                                                    'pvid', 'LHCbIDCount', 'first_hit_Z', 'last_hit_Z', 'EvtNum'])
    container[:] = []
    return df
    
    


In [5]:

def getzerr(row):
    #print(row)
    return row['pvz'] - row['z']


In [6]:



prefix="data/train"
trainfiles = os.listdir(prefix)[:3000]
testfiles = os.listdir(prefix)[3000:4000]

df = load_data(trainfiles)
df_test = load_data(testfiles)

# Adding the error
df['zerr'] = df.apply(lambda row: getzerr(row), axis=1)
df_test['zerr'] = df_test.apply(lambda row: getzerr(row), axis=1)



In [7]:
# We decide that the cutoff is at 3mm
# Adding the error


def isvalid(row):
    return (float(abs(row['zerr']) < 3.0))

df['valid'] = df.apply(lambda row: isvalid(row), axis=1)
df_test['valid'] = df.apply(lambda row: isvalid(row), axis=1)

In [8]:
df[df['valid'] == 0]

,x,y,z,tx,ty,qp,pvx,pvy,pvz,pvid,LHCbIDCount,first_hit_Z,last_hit_Z,EvtNum,zerr,valid
0,0.001914,-0.006409,-22.309256,0.003983,-0.010017,0.0,0.0232,-0.0150,-13.2077,0,5,500.580994,750.580994,559230,9.101556,0.0
1,-0.038806,-0.050898,-19.987844,0.004090,-0.009627,0.0,0.0232,-0.0150,-13.2077,0,4,600.580994,750.580994,559230,6.780144,0.0
2,0.145686,0.198580,133.406631,0.028982,-0.016534,0.0,-0.0116,0.0082,89.5389,737,4,600.580994,749.419006,559230,-43.867731,0.0
4,1.617633,0.889093,37.046951,-0.033355,0.062746,0.0,0.0232,-0.0150,-13.2077,0,4,249.419006,499.419006,559230,-50.254651,0.0
7,-0.992756,0.002872,75.253731,-0.000304,-0.106804,0.0,-0.0116,0.0082,89.5389,737,4,261.919006,488.080994,559230,14.285169,0.0
8,1.876935,0.587155,42.906895,0.036616,-0.073036,0.0,-0.0122,0.0132,-53.2394,483,11,100.581001,486.919006,559230,-96.146295,0.0
12,2.313233,-1.173654,-27.497160,0.032765,0.074237,0.0,0.0232,-0.0150,-13.2077,0,5,225.580994,400.580994,559230,14.289460,0.0
14,-0.028335,0.058264,85.842728,-0.014528,0.099869,0.0,-0.0116,0.0082,89.5389,737,7,163.080994,399.419006,559230,3.696172,0.0
20,1.614120,3.316390,108.367393,-0.086970,0.046845,0.0,0.0232,-0.0150,-13.2077,0,7,163.080994,386.919006,559230,-121.575093,0.0
24,-0.029226,-0.027234,95.594833,0.014194,-0.006416,0.0,-0.0116,0.0082,89.5389,737,5,500.580994,750.580994,559230,-6.055933,0.0


Trying a random forest classifier
===========================

In [9]:

train_columns = ['z', 'tx', 'ty', 'first_hit_Z', 'last_hit_Z']
X = df.loc[:, train_columns].values
y = df.loc[:, 'valid'].values

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from scipy import interp

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=6)

plt.rcParams['figure.figsize'] = (15, 6)


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)




In [44]:

classifiers = []
random_state = np.random.RandomState(0)

i = 0
for train, test in cv.split(X, y):
    cl = RandomForestClassifier(n_estimators=10, max_depth=3, min_samples_split=2, random_state=random_state)
    probas_ = cl.fit(X[train], y[train]).predict_proba(X[test])
    probas_train = cl.predict_proba(X[train])
    classifiers.append(cl)
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    fpr_train, tpr_train, thresholds_train = roc_curve(y[train], probas_train[:, 1])
    plt.plot(fpr_train, tpr_train, lw=1, alpha=0.3,
             label='ROC fold TRAIN %d (AUC = %0.2f)' % (i, auc(fpr_train, tpr_train)))

    i += 1
    
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right");



In [45]:
X_test = df_test.loc[:, train_columns].values
y_test = df_test.loc[:, 'valid'].values

from sklearn.metrics import recall_score, precision_score, f1_score

recall_scores = []
precision_scores = []
f1_scores = []
for c in classifiers:
    y_pred = c.predict(X_test)
    recall_scores.append(recall_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

def stats(l):
    return (np.mean(l), np.std(l))
    
print("recall_scores", stats(recall_scores))
print("precision_scores", stats(precision_scores))
print("f1_scores", stats(f1_scores))


recall_scores (0.6655994965145949, 0.04757056846512561)
precision_scores (0.6553610745439441, 0.000347259744891035)
f1_scores (0.6596159238519426, 0.02301155989543916)


In [46]:

print(recall_scores)

[0.6625177006587724, 0.607630266587313, 0.7542738112340182, 0.6190613007162353, 0.6750672111970776, 0.6750466886941532]
